In [1]:
# Importing dependencies 
import torch
from PIL import Image
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Loading Data

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Graustufe
    transforms.Resize((128, 128)),  # Größe Bild
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # guter Mittelwert/ std für Normalisierung?
])
train_dataset = datasets.ImageFolder(root="archive/Training", transform=transform)      #pituitary aus Datensatz rausgelöscht
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [3]:
# Define the image classifier model
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3), 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 14 * 14, 3)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [4]:
# Create an instance of the image classifier model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
classifier = ImageClassifier().to(device)

cpu


In [13]:
# Define the optimizer and loss function
optimizer = Adam(classifier.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [14]:
# Train the model
for epoch in range(100):  
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = classifier(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
    print(f"Epoch:{epoch} loss is {loss.item()}")


Epoch:0 loss is 0.5434936881065369
Epoch:1 loss is 0.41650325059890747
Epoch:2 loss is 0.21147534251213074
Epoch:3 loss is 0.13654690980911255
Epoch:4 loss is 0.19535963237285614
Epoch:5 loss is 0.04896220192313194
Epoch:6 loss is 0.021450968459248543
Epoch:7 loss is 0.12505456805229187
Epoch:8 loss is 0.07503528892993927
Epoch:9 loss is 0.004126415122300386
Epoch:10 loss is 0.007745769340544939
Epoch:11 loss is 0.0035514095798134804
Epoch:12 loss is 0.0018548954976722598
Epoch:13 loss is 0.004341718275099993
Epoch:14 loss is 0.002759074093773961
Epoch:15 loss is 0.0018861220451071858
Epoch:16 loss is 0.0006715499912388623
Epoch:17 loss is 0.0002431365428492427
Epoch:18 loss is 0.00012282149691600353
Epoch:19 loss is 0.0005385594558902085
Epoch:20 loss is 0.00021837090025655925
Epoch:21 loss is 0.00022135984909255058
Epoch:22 loss is 6.30414069746621e-05
Epoch:23 loss is 8.93718097358942e-05
Epoch:24 loss is 0.00015359267126768827
Epoch:25 loss is 0.00018932415696326643
Epoch:26 loss i

In [15]:
# Save the trained model
torch.save(classifier.state_dict(), 'model_state.pt')

In [16]:
# Load the saved model
with open('model_state.pt', 'rb') as f: 
     classifier.load_state_dict(load(f))  
       

In [17]:
#Testing

import os
#test_pfad = 'archive/Testing'


img_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_dataset = datasets.ImageFolder(root="archive/Testing", transform=img_transform)      #pituitary aus Datensatz rausgelöscht
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False) 

#dataiter = iter(test_loader)
#images, labels = next(dataiter)

#img_tensor = img_transform(images).unsqueeze(0).to(device)
class_names = train_dataset.classes  

with torch.no_grad():
    for idx, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = classifier(images)
        predicted_index = torch.argmax(outputs, dim=1).item()
        true_index = labels.item()

        predicted_label = class_names[predicted_index]
        true_label = class_names[true_index]

        print(f"[{idx+1}] Predicted: {predicted_label} | True: {true_label}")
      

#Klassennamen laden
#class_names = train_dataset.classes        
#predicted_label = class_names[predicted_index]
#print(f"Predicted class: {predicted_label}")


[1] Predicted: glioma | True: glioma
[2] Predicted: glioma | True: glioma
[3] Predicted: glioma | True: glioma
[4] Predicted: glioma | True: glioma
[5] Predicted: glioma | True: glioma
[6] Predicted: glioma | True: glioma
[7] Predicted: glioma | True: glioma
[8] Predicted: glioma | True: glioma
[9] Predicted: glioma | True: glioma
[10] Predicted: glioma | True: glioma
[11] Predicted: glioma | True: glioma
[12] Predicted: glioma | True: glioma
[13] Predicted: glioma | True: glioma
[14] Predicted: glioma | True: glioma
[15] Predicted: glioma | True: glioma
[16] Predicted: glioma | True: glioma
[17] Predicted: glioma | True: glioma
[18] Predicted: glioma | True: glioma
[19] Predicted: glioma | True: glioma
[20] Predicted: meningioma | True: glioma
[21] Predicted: glioma | True: glioma
[22] Predicted: glioma | True: glioma
[23] Predicted: glioma | True: glioma
[24] Predicted: glioma | True: glioma
[25] Predicted: glioma | True: glioma
[26] Predicted: glioma | True: glioma
[27] Predicted: g